In [1]:
import copy
import random

import numpy as np
import pandas as pd
import torch
from scipy import stats
from torch import nn
from torchtext.legacy import data
from torchtext.vocab import Vectors
from tqdm import tqdm

from util import calc_accuracy, calc_f1, init_device, load_params
from util.mmd import run_test
from util.model import MMD, MyEmbedding, MyMLP
from util.nlp_preprocessing import dataframe2dataset, tokenizer_ja

In [2]:
# ランダムシード初期化
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
device = init_device()

# パラメータ読み込み
print("Loading parameters...")
params = load_params("/workspace/amazon_review/config/params_mmd.json")

# データセット読み込み
train_df = pd.read_json(params["ja_train_path"], orient="record", lines=True)
if params["is_developing"]:
    train_df = train_df.sample(n=200000, random_state=1)
dev_df = pd.read_json(params["ja_dev_path"], orient="record", lines=True)
test_df = pd.read_json(params["ja_test_path"], orient="record", lines=True)

# sourceカテゴリーとtargetカテゴリーを分ける
train_source_df = train_df[train_df["product_category"] == params["source_category"]]
dev_source_df = dev_df[dev_df["product_category"] == params["source_category"]]
test_source_df = test_df[test_df["product_category"] == params["source_category"]]
train_target_df = train_df[train_df["product_category"] == params["target_category"]]
dev_target_df = dev_df[dev_df["product_category"] == params["target_category"]]
test_target_df = test_df[test_df["product_category"] == params["target_category"]]

# クラスラベル設定
for df in [train_source_df, dev_source_df, test_source_df, train_target_df, dev_target_df, test_target_df]:
    # 3以上かを予測する場合
    df["class"] = 0
    df["class"][df["stars"] > 3] = 1

    # 5クラス分類する場合
    # df["class"] = df["stars"] - 1

# フィールド作成
print("Building data iterator...")
text_field = data.Field(
    sequential=True,
    tokenize=tokenizer_ja,
    use_vocab=True,
    lower=True,
    include_lengths=True,
    batch_first=True,
    fix_length=params["token_max_length"],
    init_token="<cls>",
    eos_token="<eos>",
)
label_field = data.Field(sequential=False, use_vocab=False)
fields = [("text", text_field), ("label", label_field)]

# データセット作成
columns = ["review_body", "class"]
train_source_dataset = dataframe2dataset(train_source_df, fields, columns)
dev_source_dataset = dataframe2dataset(dev_source_df, fields, columns)
test_source_dataset = dataframe2dataset(test_source_df, fields, columns)
# train_target_dataset = dataframe2dataset(train_target_df, fields, columns)
dev_target_dataset = dataframe2dataset(dev_target_df, fields, columns)
test_target_dataset = dataframe2dataset(test_target_df, fields, columns)
all_train_dataset = dataframe2dataset(pd.concat([train_source_df, train_target_df]), fields, columns)

# embedding作成
if params["use_pretrained_vector"]:
    japanese_fasttext_vectors = Vectors(name=params["ja_vector_path"])
    text_field.build_vocab(all_train_dataset, vectors=japanese_fasttext_vectors, min_freq=1)
else:
    text_field.build_vocab(all_train_dataset, min_freq=1)

# データローダー作成
dev_source_iter = data.BucketIterator(
    dataset=dev_source_dataset, batch_size=params["batch_size"], train=False, sort=False
)
test_source_iter = data.BucketIterator(
    dataset=test_source_dataset, batch_size=params["batch_size"], train=False, sort=False
)
# train_target_iter = data.BucketIterator(dataset=train_target_dataset, batch_size=params["batch_size"], train=True)
dev_target_iter = data.BucketIterator(
    dataset=dev_target_dataset, batch_size=params["batch_size"], train=False, sort=False
)
test_target_iter = data.BucketIterator(
    dataset=test_target_dataset, batch_size=params["batch_size"], train=False, sort=False
)

# モデル構築
v_size = len(text_field.vocab.stoi)
if params["use_pretrained_vector"]:
    source_embedding = MyEmbedding(
        params["emb_dim"], v_size, params["token_max_length"], params["class_num"], text_field
    ).to(device)
    target_embedding = MyEmbedding(
        params["emb_dim"], v_size, params["token_max_length"], params["class_num"], text_field
    ).to(device)
else:
    source_embedding = MyEmbedding(params["emb_dim"], v_size, params["token_max_length"]).to(device)
    target_embedding = MyEmbedding(params["emb_dim"], v_size, params["token_max_length"]).to(device)
mlp = MyMLP(class_num=params["class_num"]).to(device)
criterion = getattr(nn, params["criterion"])()
mmd = MMD("rbf")

optimizer_s_emb = getattr(torch.optim, params["optimizer"])(source_embedding.parameters(), lr=params["lr"])
optimizer_t_emb = getattr(torch.optim, params["optimizer"])(target_embedding.parameters(), lr=params["lr"])
optimizer_mlp = getattr(torch.optim, params["optimizer"])(mlp.parameters(), lr=params["lr"])

GPU available: cuda
Loading parameters...
 ja_train_path:	 /workspace/data/dataset_ja_train.json
 ja_dev_path:	 /workspace/data/dataset_ja_dev.json
 ja_test_path:	 /workspace/data/dataset_ja_test.json
 en_train_path:	 /workspace/data/dataset_en_train.json
 en_dev_path:	 /workspace/data/dataset_en_dev.json
 en_test_path:	 /workspace/data/dataset_en_test.json
 ja_vector_path:	 /workspace/amazon_review/weight/japanese_fasttext_vectors.vec
 is_developing:	 True
 source_category:	 home
 target_category:	 wireless
 target_ratio:	 0.5
 lambda:	 1.0
 use_pretrained_vector:	 False
 token_max_length:	 256
 batch_size:	 32
 emb_dim:	 300
 class_num:	 2
 criterion:	 CrossEntropyLoss
 lr:	 1e-05
 optimizer:	 Adam
 epochs:	 10
 trial_count:	 10


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Building data iterator...


In [3]:
params["target_ratio"] = [0.01, 0.05, 0.1, 0.3, 0.5]

for target_ratio in params["target_ratio"]:
    print("------------------------------")
    print(f"target_ratio = {target_ratio}")
    print("------------------------------")

    source_accuracy_list = []
    source_f1_list = []
    target_accuracy_list = []
    target_f1_list = []

    # targetでFineTuningする準備
    # target_ratioで指定した比率までtargetのデータ数を減らす
    source_num = train_source_df.shape[0]
    target_num = int(source_num * target_ratio)
    if target_num > train_target_df.shape[0]:
        print("Target ratio is too large.")
        exit()

    for count in range(params["trial_count"]):
        print(f"\n{count+1}回目の試行")

        # targetのデータローダー作成
        train_target_df_sample = train_target_df.sample(target_num, replace=False)
        print(f"Source num: {source_num}, Target num: {target_num}")
        train_target_dataset = dataframe2dataset(train_target_df_sample, fields, columns)
        train_target_iter = data.BucketIterator(
            dataset=train_target_dataset, batch_size=params["batch_size"], train=True
        )

        # モデル初期化
        v_size = len(text_field.vocab.stoi)
        if params["use_pretrained_vector"]:
            source_embedding = MyEmbedding(
                params["emb_dim"], v_size, params["token_max_length"], params["class_num"], text_field
            ).to(device)
            target_embedding = MyEmbedding(
                params["emb_dim"], v_size, params["token_max_length"], params["class_num"], text_field
            ).to(device)
        else:
            source_embedding = MyEmbedding(params["emb_dim"], v_size, params["token_max_length"]).to(device)
            target_embedding = MyEmbedding(params["emb_dim"], v_size, params["token_max_length"]).to(device)
        mlp = MyMLP(class_num=params["class_num"]).to(device)
        criterion = getattr(nn, params["criterion"])()
        mmd = MMD("rbf")

        optimizer_s_emb = getattr(torch.optim, params["optimizer"])(source_embedding.parameters(), lr=params["lr"])
        optimizer_t_emb = getattr(torch.optim, params["optimizer"])(target_embedding.parameters(), lr=params["lr"])
        optimizer_mlp = getattr(torch.optim, params["optimizer"])(mlp.parameters(), lr=params["lr"])

        split_ratio = len(train_target_dataset) / len(train_source_dataset)
        
        # 訓練
        for epoch in range(params["epochs"]):
            print(f"\nepoch {epoch+1} / {params['epochs']}")
            # 各lossの初期化
            total_source_loss = 0
            total_target_loss = 0
            total_mmd_loss = 0
            total_all_loss = 0

            # MMDのためにターゲットと同数のソースが必要なためエポックごとにソースのデータローダーを作成し直す
            # エポックごとに異なるソースのデータが使用されるようになる
            random.seed(epoch)
            train_source_subset, _ = train_source_dataset.split(split_ratio=split_ratio)
            train_source_iter = data.BucketIterator(
                dataset=train_source_subset, batch_size=params["batch_size"], train=True
            )

            for i, (source_batch, target_batch) in enumerate(zip(train_source_iter, train_target_iter)):
                source_embedding.train()
                target_embedding.train()
                mlp.train()

                source_x, source_y = source_batch.text[0].to(device), (source_batch.label).to(device)
                target_x, target_y = target_batch.text[0].to(device), (target_batch.label).to(device)

                # MMDの処理はBatch数が同数でなけらばならないためcontinueする
                # sourceとtargetで毎回同じBatch数だけデータがロードされる処理ができれば下の処理は不要
                if source_x.shape[0] != params["batch_size"] or target_x.shape[0] != params["batch_size"]:
                    continue

                source_embed = source_embedding(source_x)
                source_pred = mlp(source_embed)
                source_loss = criterion(source_pred, source_y)
                total_source_loss += source_loss.cpu()

                target_embed = target_embedding(target_x)
                target_pred = mlp(target_embed)
                target_loss = criterion(target_pred, target_y)
                total_target_loss += target_loss.cpu()

                if params["lambda"] == 0:
                    all_loss = source_loss + target_loss
                else:
                    mmd_loss = mmd(source_embed, target_embed)
                    total_mmd_loss += mmd_loss.cpu()
                    all_loss = source_loss + target_loss + params["lambda"] * mmd_loss

                optimizer_s_emb.zero_grad()
                optimizer_t_emb.zero_grad()
                optimizer_mlp.zero_grad()

                all_loss.backward()

                optimizer_s_emb.step()
                optimizer_t_emb.step()
                optimizer_mlp.step()
                total_all_loss += all_loss.cpu()

            mean_source_loss = total_source_loss / len(train_source_iter)
            mean_target_loss = total_target_loss / len(train_target_iter)
            mean_all_loss = total_all_loss / len(train_source_iter)
            if params["lambda"] == 0:
                print(f"Loss -> Source: {mean_source_loss:.3f}\tTarget: {mean_target_loss:.3f}\tAll: {mean_all_loss:.3f}")
            else:
                mean_mmd_loss = total_mmd_loss / len(train_source_iter)
                print(
                    f"Loss -> Source: {mean_source_loss:.3f}\tTarget: {mean_target_loss:.3f}\tMMD: {mean_mmd_loss:.3f}\tAll: {mean_all_loss:.3f}"  # noqa #E501
                )

            # dev_source_accuracy, dev_source_f1 = run_test(source_embedding, mlp, dev_source_iter, device)
            # print(f"\nDev source Accuracy: {dev_source_accuracy:.2f}")
            # print(f"Dev source F1 Score: {dev_source_f1:.2f}")
            # dev_target_accuracy, dev_target_f1 = run_test(target_embedding, mlp, dev_target_iter, device)
            # print(f"\nDev target Accuracy: {dev_target_accuracy:.2f}")
            # print(f"Dev target F1 Score: {dev_target_f1:.2f}")

        test_source_accuracy, test_source_f1 = run_test(source_embedding, mlp, test_source_iter, device)
        test_target_accuracy, test_target_f1 = run_test(target_embedding, mlp, test_target_iter, device)
        source_accuracy_list.append(test_source_accuracy)
        source_f1_list.append(test_source_f1)
        target_accuracy_list.append(test_target_accuracy)
        target_f1_list.append(test_target_f1)
        print(f"\nTest Source Accuracy: {test_source_accuracy:.2f}")
        print(f"Test Source F1 Score: {test_source_f1:.2f}")
        print(f"Test Target Accuracy: {test_target_accuracy:.2f}")
        print(f"Test Target F1 Score: {test_target_f1:.2f}")

    # sourceの信頼区間
    source_accuracy_interval = stats.t.interval(
        alpha=0.95, df=len(source_accuracy_list) - 1, loc=np.mean(source_accuracy_list), scale=stats.sem(source_accuracy_list)
    )
    source_f1_interval = stats.t.interval(alpha=0.95, df=len(source_f1_list) - 1, loc=np.mean(source_f1_list), scale=stats.sem(source_f1_list))
    target_accuracy_interval = stats.t.interval(
        alpha=0.95, df=len(target_accuracy_list) - 1, loc=np.mean(target_accuracy_list), scale=stats.sem(target_accuracy_list)
    )
    target_f1_interval = stats.t.interval(alpha=0.95, df=len(target_f1_list) - 1, loc=np.mean(target_f1_list), scale=stats.sem(target_f1_list))
    
    print("\n\tSource: Mean, Std, 95% interval (bottom, up)")
    print(
        f"Accuracy\t{np.mean(source_accuracy_list):.2f}, {np.std(source_accuracy_list, ddof=1):.2f}, {source_accuracy_interval[0]:.2f}, {source_accuracy_interval[1]:.2f}"
    )
    print(
        f"F1 Score\t{np.mean(source_f1_list):.2f}, {np.std(source_f1_list, ddof=1):.2f}, {source_f1_interval[0]:.2f}, {source_f1_interval[1]:.2f}"
    )
    print("\n\tTarget: Mean, Std, 95% interval (bottom, up)")
    print(
        f"Accuracy\t{np.mean(target_accuracy_list):.2f}, {np.std(target_accuracy_list, ddof=1):.2f}, {target_accuracy_interval[0]:.2f}, {target_accuracy_interval[1]:.2f}"
    )
    print(
        f"F1 Score\t{np.mean(target_f1_list):.2f}, {np.std(target_f1_list, ddof=1):.2f}, {target_f1_interval[0]:.2f}, {target_f1_interval[1]:.2f}"
    )

------------------------------
target_ratio = 0.01
------------------------------

1回目の試行
Source num: 18662, Target num: 186

epoch 1 / 10
Loss -> Source: 0.638	Target: 0.568	MMD: 0.796	All: 2.002

epoch 2 / 10
Loss -> Source: 0.497	Target: 0.329	MMD: 0.424	All: 1.249

epoch 3 / 10
Loss -> Source: 0.469	Target: 0.270	MMD: 0.363	All: 1.101

epoch 4 / 10
Loss -> Source: 0.467	Target: 0.233	MMD: 0.341	All: 1.041

epoch 5 / 10
Loss -> Source: 0.439	Target: 0.202	MMD: 0.315	All: 0.956

epoch 6 / 10
Loss -> Source: 0.451	Target: 0.167	MMD: 0.318	All: 0.936

epoch 7 / 10
Loss -> Source: 0.487	Target: 0.155	MMD: 0.302	All: 0.945

epoch 8 / 10
Loss -> Source: 0.465	Target: 0.134	MMD: 0.286	All: 0.885

epoch 9 / 10
Loss -> Source: 0.450	Target: 0.134	MMD: 0.269	All: 0.853

epoch 10 / 10
Loss -> Source: 0.462	Target: 0.113	MMD: 0.281	All: 0.856

Test Source Accuracy: 51.11
Test Source F1 Score: 34.58
Test Target Accuracy: 54.48
Test Target F1 Score: 34.53

2回目の試行
Source num: 18662, Target num: 18